# Interactive Dashboard App

**Project Goal:** To build a dynamic web application that will allow users to choose their own features, build a model, and evaluate its performance through a graphic user interface - a tool that will allow anyone to build a model without code.

**Specific Goals:**
- Import and clean dataset
- Build web application layout
- Build interactive elements
- Launch Application.


Build Application:
- Application Layout
- Variance bar chart: Call back, decorator
- K-Means slider
- PCA scatter plot

In [69]:
import pandas as pd 
import plotly.express as px
from dash import Input, Output, dcc, html
from jupyter_dash import JupyterDash
from dash import Dash
from scipy.stats.mstats import trimmed_var 
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

### Prepare Data

Import

- Creating a wrangle function to import the data:

In [70]:
def wrangle(filepath):

    """Read SCF data file into ``DataFrame``.

    Returns only credit fearful households whose net worth is less than $2 million.

    Parameters
    ----------
    filepath : str
        Location of CSV file.
    """
    df = pd.read_csv(filepath)
    
    mask = (df["TURNFEAR"]==1) & (df["NETWORTH"]< 2e6)
    df = df[mask]
    
    return df

- Using the wrangle function to import the data:

In [71]:
df=wrangle("data/customer.csv")
print("df type:", type(df))
print("df shape:", df.shape)
df.head()

df type: <class 'pandas.core.frame.DataFrame'>
df shape: (4418, 351)


,YY1,Y1,WGT,HHSEX,AGE,AGECL,EDUC,EDCL,MARRIED,KIDS,...,NWCAT,INCCAT,ASSETCAT,NINCCAT,NINC2CAT,NWPCTLECAT,INCPCTLECAT,NINCPCTLECAT,INCQRTCAT,NINCQRTCAT
5,2,21,3790.476607,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2
6,2,22,3798.868505,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,3,2,2
7,2,23,3799.468393,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2
8,2,24,3788.076005,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2
9,2,25,3793.066589,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2


### Build Dashboard

#### Application Layout

Instatntiate Application

In [72]:
app = Dash(__name__)

print("app type:", type(app))

app type: <class 'dash.dash.Dash'>


Variant Bar Chart

Bar Chart: Presentation Layer

A function that returns the 5 highest variance features in the df.

- Step 1 is to create a get_high_var_features function that returns the five highest-variance features in a DataFrame. 

In [73]:
def get_high_var_features(trimmed=True, return_feat_names=True):

    """Returns the five highest-variance features of ``df``.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.

    return_feat_names : bool, default=False
        If ``True``, returns feature names as a ``list``. If ``False``
        returns ``Series``, where index is feature names and values are
        variances.
    """
    # Calculate variance
    if trimmed:
        top_five_features = (
        df.apply(trimmed_var).sort_values().tail(5)
        )
    else:
        top_five_features = df.var().sort_values().tail(5)
    
    # Extract names
    if return_feat_names:
        top_five_features = top_five_features.index.tolist()
    
    return top_five_features

In [74]:
get_high_var_features()

['DEBT', 'NETWORTH', 'HOUSES', 'NFIN', 'ASSET']

In [75]:
get_high_var_features(trimmed=False)

['NHNFIN', 'HOUSES', 'NETWORTH', 'NFIN', 'ASSET']

In [76]:
get_high_var_features(return_feat_names=False)

DEBT        3.089865e+09
NETWORTH    3.099929e+09
HOUSES      4.978660e+09
NFIN        8.456442e+09
ASSET       1.175370e+10
dtype: float64

Bar Chart: Service Layer

- We need to create a serve_bar_chart function that returns a plotly express bar chart of the five highest-variance features.

In [77]:

def serve_bar_chart(trimmed=True):

    """Returns a horizontal bar chart of five highest-variance features.

    Parameters
    ----------
    trimmed : bool, default=True
        If ``True``, calculates trimmed variance, removing bottom and top 10%
        of observations.
    """
    # Get features
    top_five_features = get_high_var_features(trimmed=trimmed, return_feat_names=False)
    
    #Build bar chart
    fig = px.bar(
        x=top_five_features,
        y=top_five_features.index,
        orientation="h"
    )
    fig.update_layout(
        xaxis_title="Variance",
        yaxis_title="Feature"
    )
    
    return fig

In [78]:
serve_bar_chart()

In [79]:
serve_bar_chart(trimmed=False)

Bar Chart: Add Chart to Presentation Layer

Layout

In [80]:
app.layout = html.Div(
    [
        # Application Title
        html.H1("Survey of Consumer Finances"),
        
        # Bar Chart Element
        html.H2("High Variance Features"), 

        # Bar chart
        dcc.Graph(figure=serve_bar_chart(),id="bar-chart"),
        dcc.RadioItems(
            options=[
                {"label": "trimmed", "value": True},
                {"label": "not trimmed", "value": False}
            ],
            value=True,
            id="trim-buttom"
        )
    ]

)

### Application Deployment

In [81]:
app.run_server(debug=True)